已经拿到col4了， 做好它，这样剩下的都可以同样操作方式了；

In [3]:
import json
import scipy
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from scipy import sparse
import pickle
import torch
from torch_geometric.data import Data

In [2]:
oss_js_root = '/data-0/qibo/Gdata/nodefeats/origin/'
save_root = '/data-0/qibo/Gdata/col_4/'
feat_root = '/data-0/qibo/Gdata/origin/'

In [4]:
with open(oss_js_root+'col_4_js', 'r') as f:
    col = json.load(f)

In [3]:
def csr_save_feats(col_num):
    col_path = oss_js_root + 'col_{}_js'.format(col_num)
    with open(col_path, 'r') as f:
        col = json.load(f)
    col_names = pd.DataFrame.from_dict(col).iloc[:,:-1].columns
    col_names = [i.split('_')[1] for i in col_names]
    col_values = pd.DataFrame.from_dict(col).iloc[:,:-1].values
    csr_col = csr_matrix(col_values)
    sparse.save_npz(save_root + 'feat.npz', csr_col)
    with open(save_root+'col_name.pkl', 'wb') as f:
        pickle.dump(col_names, f)
    print('csr file saved in {}{}'.format(save_root, 'feat.npz'))
    print('col_names saved in {}{}'.format(save_root, 'col_name.pkl'))
    return None

def csr_save_edge(col_num):
    edge_path = edge_root + 'col_{}.csv'.format(col_num)
    edges = pd.read_csv(edge_path, header=None).iloc[:,:2].values
    csr_edges = csr_matrix(edges)
    sparse.save_npz(save_root + 'edges.npz', csr_edges)
    print('csr file saved in {}{}'.format(save_root, 'edges.npz'))
    return 

def csr_save_label(y):
    csr_labels = csr_matrix(y.values)
    sparse.save_npz(save_root + 'y.npz', csr_labels)
    print('csr file saved in {}{}'.format(save_root, 'y.npz'))
    return 

1. 数据压缩 存到 save_root; for node feats;

In [ ]:
csr_save_feats(4)

2. for edges;

In [ ]:
csr_save_edge(4)

3. load 测试；

In [ ]:
x = sparse.load_npz("/data-0/qibo/Gdata/col_4/feat.npz")
edges = sparse.load_npz("/data-0/qibo/Gdata/col_4/edges.npz")
with open('/data-0/qibo/Gdata/col_4/col_name.pkl', 'rb') as f:
    col_names = pickle.load(f)

4. label；n_overdue == n_fund ; 因为这是18年数据，不可能还有没结清的；

In [9]:
df = pd.DataFrame(x.toarray(), columns=col_names)
y = df.fund == df.paidoff
y = y*1 

In [10]:
csr_save_label(y)

csr file saved in /data-0/qibo/Gdata/col_4/y.npz


5. load all 

In [1]:
root = '/data-0/qibo/Gdata/col_4/raw'

In [53]:
oss_js_root = '/data-0/qibo/Gdata/nodefeats/origin/'
save_root = '/data-0/qibo/Gdata/col_4/'
edge_root = '/data-0/qibo/Gdata/origin/'
def load_all():
    dic_path = oss_js_root + 'col_4_dict'
    with open(dic_path, 'r') as f:
        userid2idx = json.load(f)
    x = sparse.load_npz("/data-0/qibo/Gdata/col_4/feat.npz")
    y = sparse.load_npz("/data-0/qibo/Gdata/col_4/y.npz")
    with open('/data-0/qibo/Gdata/col_4/col_name.pkl', 'rb') as f:
        col_names = pickle.load(f)
    x = torch.tensor(x.toarray(), dtype =torch.float)
    y = torch.tensor(y.toarray()).squeeze()
    edges = sparse.load_npz("/data-0/qibo/Gdata/col_4/edges.npz").toarray().T
    row1 = [userid2idx[str(i)] for i in edges[0]]
    row2 = [userid2idx[str(i)] for i in edges[1]]
    new_edges = torch.tensor(np.stack([row1, row2]))
    return x, y, new_edges

x, y, edges = load_all()
data = Data(x=x, edge_index=edges, y=y)

In [5]:
with open(root + '/col_4_dict', 'r') as f:
    userid2idx = json.load(f)
with open(root + '/col_name.pkl', 'rb') as f:
    col_names = pickle.load(f)
# -- node_fetas;  labels; edges;
x = sparse.load_npz(root + "/feat.npz")
y = sparse.load_npz(root + "/y.npz")
edges = sparse.load_npz(root + "/edges.npz").toarray().T

df = pd.DataFrame(x.toarray(), columns=col_names)
y = df.fund == df.paidoff
y = y*1 

In [12]:
df.shape

(355965, 8)

In [16]:
df[df.fund == 0]

,apply,cancel,fund,overdue,paidoff,reject,renew,submit
0,1,0,0,0,0,1,0,1
2,1,1,0,0,0,0,0,0
7,11,0,0,0,0,11,0,11
8,9,3,0,0,0,6,0,6
9,0,0,0,0,0,0,0,0
13,1,0,0,0,0,1,0,1
15,1,0,0,0,0,1,0,1
16,1,0,0,0,0,0,0,0
17,0,0,0,0,0,0,0,0
20,1,0,0,0,0,1,0,1


In [15]:
df[(df.paidoff==df.fund)&(df.paidoff!=0)&(df.overdue==0)]

,apply,cancel,fund,overdue,paidoff,reject,renew,submit
100,10,1,3,0,3,6,47,10
215,7,2,5,0,5,0,1,5
367,8,1,7,0,7,0,0,8
402,2,0,1,0,1,0,0,1
467,9,0,9,0,9,0,5,9
555,13,0,7,0,7,6,3,13
559,1,0,1,0,1,0,1,1
570,1,0,1,0,1,0,0,1
725,1,0,1,0,1,0,0,1
815,6,0,6,0,6,0,2,6


In [8]:
df[(df.paidoff==0)&(df.fund==1)]

,apply,cancel,fund,overdue,paidoff,reject,renew,submit
41,4,0,1,1,0,3,0,4
64,2,0,1,1,0,1,0,2
68,1,0,1,1,0,0,0,1
75,3,2,1,1,0,0,0,2
90,11,2,1,1,0,8,0,9
102,1,0,1,1,0,0,0,1
116,4,3,1,1,0,0,0,4
117,1,0,1,1,0,0,0,1
123,1,0,1,1,0,0,0,1
137,1,0,1,1,0,0,0,1


In [9]:
df[(df.paidoff==0)&(df.fund==0)]

,apply,cancel,fund,overdue,paidoff,reject,renew,submit
0,1,0,0,0,0,1,0,1
2,1,1,0,0,0,0,0,0
7,11,0,0,0,0,11,0,11
8,9,3,0,0,0,6,0,6
9,0,0,0,0,0,0,0,0
13,1,0,0,0,0,1,0,1
15,1,0,0,0,0,1,0,1
16,1,0,0,0,0,0,0,0
17,0,0,0,0,0,0,0,0
20,1,0,0,0,0,1,0,1


6. InmemeryDataset 对象；

In [41]:
import json
import scipy
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from scipy import sparse
import pickle
import torch
from torch_geometric.data import Data

import torch
import torch.nn.functional as F
# from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import SplineConv,GCNConv


import torch
from torch_geometric.data import InMemoryDataset, download_url
# from torch_geometric.read import read_planetoid_data
def sample_mask(index, num_nodes):
    mask = torch.zeros((num_nodes, ), dtype=torch.uint8)
    mask[index] = 1
    return mask

def split_idx(y, split):
    if isinstance(y, torch.Tensor):
        y = torch.tensor(y)
    x_perm = y[torch.randperm(len(y))]
    train_end_idx = int(len(y)*split[0])
    valid_end_idx = int(len(y)*(split[0]+split[1]))
    train_idx = x_perm[:train_end_idx]
    valid_idx = x_perm[train_end_idx:valid_end_idx]
    test_idx = x_perm[valid_end_idx:]
    return train_idx, valid_idx, test_idx

def samples_split_mask(y, split=(0.6,0.2)):
    train_idx, valid_idx, test_idx = split_idx(y, split)
    train_mask = sample_mask(train_idx, num_nodes=y.size(0))
    val_mask = sample_mask(valid_idx, num_nodes=y.size(0))
    test_mask = sample_mask(test_idx, num_nodes=y.size(0))
    return train_mask, val_mask, test_mask

def read_cashbus_data(root):
    # -- userid2idx;
    with open(root + '/col_4_dict', 'r') as f:
        userid2idx = json.load(f)
    # -- node_fetas;  labels; edges;
    x = sparse.load_npz(root + "/feat.npz")
    y = sparse.load_npz(root + "/y.npz")
    edges = sparse.load_npz(root + "/edges.npz").toarray().T
    
    x = torch.tensor(x.toarray(), dtype =torch.float)
    y = torch.tensor(y.toarray()).squeeze()
    row1 = [userid2idx[str(i)] for i in edges[0]]
    row2 = [userid2idx[str(i)] for i in edges[1]]
    new_edges = torch.tensor(np.stack([row1, row2]))
    train_mask, val_mask, test_mask = samples_split_mask(y, split=(0.6,0.2))
    data = Data(x=x, edge_index=new_edges, y=y)
    data.train_mask = train_mask
    data.val_mask = val_mask
    data.test_mask = test_mask
    return data


class CashBus(InMemoryDataset):
    r"""The citation network datasets "Cora", "CiteSeer" and "PubMed" from the
    `"Revisiting Semi-Supervised Learning with Graph Embeddings"
    <https://arxiv.org/abs/1603.08861>`_ paper.
    Nodes represent documents and edges represent citation links.
    Training, validation and test splits are given by binary masks.
    Args:
        root (string): Root directory where the dataset should be saved.
        name (string): The name of the dataset (:obj:`"Cora"`,
            :obj:`"CiteSeer"`, :obj:`"PubMed"`).
        transform (callable, optional): A function/transform that takes in an
            :obj:`torch_geometric.data.Data` object and returns a transformed
            version. The data object will be transformed before every access.
            (default: :obj:`None`)
        pre_transform (callable, optional): A function/transform that takes in
            an :obj:`torch_geometric.data.Data` object and returns a
            transformed version. The data object will be transformed before
            being saved to disk. (default: :obj:`None`)
    """


    def __init__(self, root, transform=None, pre_transform=None):
        super(CashBus, self).__init__(root, transform, pre_transform)
        print('processed_path:{}'.format(self.processed_paths))
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return ['col_4_dict', 'feat.npz', 'y.npz', 'edges.npz', 'col_name.pkl']

    @property
    def processed_file_names(self):
        return 'data.pt'
    
    def download(self):
        pass

    def process(self):
        print('go pl, raw_dir:{}'.format(self.raw_dir))
        data = read_cashbus_data(self.raw_dir)
        data = data if self.pre_transform is None else self.pre_transform(data)
        data, slices = self.collate([data])
        torch.save((data, slices), self.processed_paths[0])

    def __repr__(self):
        return '{}()'.format(self.name)